In [1]:
from bs4 import BeautifulSoup as bs
from decimal import Decimal
import requests
import pandas as pd
from datetime import datetime

In [2]:
killer_url = "https://en.wikipedia.org/wiki/List_of_serial_killers_in_the_United_States"
wiki_url ="https://en.wikipedia.org"

In [3]:
killer_list = requests.get(killer_url)
killer_soup = bs(killer_list.text , 'html.parser')

In [4]:
killer_table = killer_soup.find_all('table', class_='wikitable')[0]
links = killer_table.find_all('a')
killers = killer_table.find('tbody').find_all('tr')

In [5]:
killer = {}
killer_list = []

In [6]:
#testing something for killer victims
num_killer = len(killers)
cur_killer = 0

for killer_page in killers:
  killer = {}
  #links = killer_page.find_all('a')
  links = []
  killer_data = killer_page.find_all('td')
  if len(killer_data) >2:
    proven_victims = killer_page.find_all('td')[2].text.strip()
    estimated_victims = killer_page.find_all('td')[3].text.strip()
    links = killer_page.find_all('td')[0].find_all('a')
  else:
    proven_victims = "N/A"
    estimated_victims = "N/A"
  print(f'Getting killer {cur_killer} / {num_killer}')

  for link in links:
    killer_name = link.text
    killer_url = link.get('href')
    if (killer_name.count("[")<1):# the footnotes show a [ show don't care about them
      print(killer_name)
      the_killer_page = requests.get(wiki_url + killer_url)
      the_killer_page_soup = bs(the_killer_page.text , 'html.parser')
      killer_birth = ""
      try:
        #first check and make sure the killer isn't a city, 
        #need to add more checks here, still picking up some
        if (the_killer_page_soup.find('span', id="coordinates")):
          pass
        else:
          killer_birth = the_killer_page_soup.find('span',class_='bday').text 
          killer = {"name" : killer_name,"born":killer_birth,
                    "proven_victims":proven_victims,"estimated_victims":estimated_victims}
          killer_list.append(killer)
      except AttributeError as e:
        #print(e)
        pass
  cur_killer +=1

Getting killer 0 / 651
Getting killer 1 / 651
Ables, Tony
Getting killer 2 / 651
Adams, Edward James
Getting killer 3 / 651
Agrue, John
Getting killer 4 / 651
Albanese, Charles
Getting killer 5 / 651
Albright, Charles
Getting killer 6 / 651
Alcala, Rodney
Getting killer 7 / 651
Alix, Franklin DeWayne
Getting killer 8 / 651
Allen, Herman
Getting killer 9 / 651
Allen, Howard
Getting killer 10 / 651
Allen, Quincy
Getting killer 11 / 651
Allridge, Ronald
Getting killer 12 / 651
Amati, Tony Ray
Getting killer 13 / 651
Anderson, Stephen Wayne
Getting killer 14 / 651
Angelo, Richard
Getting killer 15 / 651
Archerd, William Dale
Getting killer 16 / 651
Archer-Gilligan, Amy
Getting killer 17 / 651
Arguelles, Roberto
Getting killer 18 / 651
Armstrong, John Eric
Getting killer 19 / 651
Atkins, Benjamin
Getting killer 20 / 651
Atkins, Joseph
Getting killer 21 / 651
Avila-Torrez, Jorge
Getting killer 22 / 651
Balaam, Anthony
Getting killer 23 / 651
Baldi, Joseph
Getting killer 24 / 651
Ball, Joe
Ge

In [7]:
 #figure out zodiac sign
def determine_sign(bday):
  bday = datetime.strptime(bday,'%Y-%m-%d')

  day_of_month = int(bday.strftime("%d"))
  month= int(bday.strftime("%m"))
  if month == 12:
	  zodiac_sign = 'sagittarius' if (day_of_month < 22) else 'capricorn'
  elif month == 1:
	  zodiac_sign = 'capricorn' if (day_of_month < 20) else 'aquarius'
  elif month == 2:
	  zodiac_sign = 'aquarius' if (day_of_month < 19) else 'pisces'
  elif month == 3:
	  zodiac_sign = 'pisces' if (day_of_month < 21) else 'aries'
  elif month == 4:
	  zodiac_sign = 'aries' if (day_of_month < 20) else 'taurus'
  elif month == 5:
	  zodiac_sign = 'taurus' if (day_of_month < 21) else 'gemini'
  elif month == 6:
	  zodiac_sign = 'gemini' if (day_of_month < 21) else 'cancer'
  elif month == 7:
	  zodiac_sign = 'cancer' if (day_of_month < 23) else 'leo'
  elif month == 8:
	  zodiac_sign = 'leo' if (day_of_month < 23) else 'virgo'
  elif month == 9:
	  zodiac_sign = 'virgo' if (day_of_month < 23) else 'libra'
  elif month == 10:
  	zodiac_sign = 'libra' if (day_of_month < 23) else 'scorpio'
  elif month == 11:
	  zodiac_sign = 'scorpio' if (day_of_month < 22) else 'sagittarius'
  else:
    zodiac_sign = "ya basic"
  return zodiac_sign


In [8]:
for killer in killer_list:
  try:
    killer['zodiac_sign'] = determine_sign(killer['born'])
  except ValueError as e:
    killer_list.remove(killer)
    print(e)
    pass

time data '1917' does not match format '%Y-%m-%d'
time data '1949' does not match format '%Y-%m-%d'
time data '1967' does not match format '%Y-%m-%d'
time data '1958' does not match format '%Y-%m-%d'
time data '1947' does not match format '%Y-%m-%d'
time data '1954' does not match format '%Y-%m-%d'
time data 'June 24, 1960' does not match format '%Y-%m-%d'
time data '1953' does not match format '%Y-%m-%d'
time data '1925' does not match format '%Y-%m-%d'


In [9]:
the_killers_not_the_band_df = pd.DataFrame(killer_list)
the_killers_not_the_band_df

,name,born,proven_victims,estimated_victims,zodiac_sign
0,"Ables, Tony",1954-12-28,4,4+,capricorn
1,"Agrue, John",1947-09-11,3,3+,virgo
2,"Albanese, Charles",1937-06-13,3,3,gemini
3,"Albright, Charles",1933-08-10,1,3,leo
4,"Alcala, Rodney",1943-08-23,8,50–230,virgo
...,...,...,...,...,...
494,"Woodfield, Randall",1950-12-26,18,44,capricorn
495,"Wright, Douglas Franklin",1940-03-25,7,7+,aries
496,"Wuornos, Aileen",1956-02-29,7,7,pisces
497,"Yates, Robert Lee",1952-05-27,13,18+,gemini


In [10]:
the_killers_not_the_band_df.to_csv("killers.csv")